
# Reconnaissance faciale via un réseau de neuron 

## Partie 2: Reconnaissance faciale en temps réel (classification binaire "admis"/"non-admis")

## Librairies

In [21]:
import pickle
import numpy as np
import cv2
from sklearn.neural_network import MLPClassifier # Importation du multi-layer perceptron via la librairie scikit-learn


## Algorithme

In [22]:
# Variable vers le dataset
DATASET_BINAIRE = "../dataset/data_binary_classification/"

with open(DATASET_BINAIRE+'/targets.pkl', 'rb') as fh:
    targets = pickle.load(fh)

with open(DATASET_BINAIRE+'/features.pkl', 'rb') as fh:
    features = pickle.load(fh)

print('Shape of visages matrix --> ', features.shape)

Shape of visages matrix -->  (30, 50, 50, 3)


# Réseau de neuron multi-couche

## Définition et paramètre de l'algorithme

Le réseau de neurones multi-couches (MLP) est un type de réseau de neurones artificiels qui est composé de plusieurs couches de neurones, avec des connexions entre les neurones de couches adjacentes. Chaque neurone est connecté à tous les neurones de la couche précédente et de la couche suivante. Les neurones de la première couche sont les entrées du réseau, et les neurones de la dernière couche sont les sorties du réseau. Les couches intermédiaires sont appelées couches cachées.

Les paramètres $w$ et $b$ désignent respectivement les poids et les biais des connexions entre les neurones. 

- Les poids $w$ sont utilisés pour multiplier les entrées des neurones

- Les biais $b$ sont utilisés pour ajouter un terme constant aux entrées des neurones. 

Les poids et les biais sont ajustés par l'algorithme d'apprentissage du réseau de neurones pour minimiser l'erreur entre les sorties du réseau et les étiquettes des visages.

## Structure de l'algorithme d'apprentissage 

L'algorithme d'apprentissage du MLP est basé sur la rétro-propagation de l'erreur. L'algorithme d'apprentissage calcule l'erreur entre les sorties du réseau et les étiquettes des visages, et ajuste les poids et les biais des connexions entre les neurones pour minimiser cette erreur.

L'algorithme d'apprentissage utilise la dérivée de l'erreur par rapport aux poids et aux biais pour ajuster les poids et les biais des connexions entre les neurones.

La fonction d'agrégation est utilisée pour combiner les entrées des neurones en une sortie du réseau. Elle est appliquée à la somme pondérée des entrées des neurones, et produit la sortie du neurone.

La fonction d'activation est utilisée pour introduire une non-linéarité dans le réseau de neurones. La fonction d'activation est appliquée à la sortie de la fonction d'agrégation, et produit la sortie du neurone. La fonction d'activation est généralement une fonction non linéaire, telle que la fonction sigmoïde ou la fonction tangente hyperbolique.



In [23]:
"""
On entraine ici un réseau de neurones pour la reconnaissance de visages
paramètres du réseau de neurones:
- 2 couches cachées de 128 neurones
- 40 itérations
- solver: descente de gradient stochastique
- taille du batch: 64 (nombre d'images à traiter en même temps)
"""

N = len(targets)

features = features.reshape(N, -1)

mlp = MLPClassifier(
    hidden_layer_sizes = (128, 128),
    batch_size = 64,
    max_iter = 40,
    solver = "sgd"
)

mlp = MLPClassifier()
mlp.fit(features, targets)


MLPClassifier()

## Exécution

In [24]:
# On importe coordonées des visages
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe
authentifications = [] # Liste pour stocker les résultats de l'authentification
while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = mlp.predict(visage_redimensionne).astype(str)
            


            if texte[0] == '0':
                authentifications.append(0) # On stocke le résultat de l'authentification dans la liste
                texte[0] = 'Non Admis'
                cv2.putText(trame, texte[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)
            else:
                authentifications.append(1)
                texte[0] = 'Admis'
                cv2.putText(trame, texte[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.rectangle(trame, (x, y), (x + l, y + l), (10, 255, 10), 2)
      

        cv2.imshow('Reconnaissance faciale en temps réel via un MLP', trame)
        
        if cv2.waitKey(1) == 27 or len(authentifications) >= 100:
            cv2.destroyAllWindows()
            camera.release()
            break


if sum(authentifications) / len(authentifications) > 0.8: # Si plus de 80% des résultats sont positifs, on renvoie un token d'authentification OK
    print({"token": True})
else:
    print({"token": False}) # Sinon, on renvoie un token d'authentification NOK



{'token': False}
